<a href="https://colab.research.google.com/github/AlafateABULIMITI/MWEIs/blob/master/mwei/training_seq2seq_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install tensorflow==1.12.0
!pip install keras==2.2.4
!git clone https://www.github.com/farizrahman4u/recurrentshop.git
%cd /content/recurrentshop
!python3 setup.py install
!sudo pip install git+https://github.com/farizrahman4u/seq2seq.git
%matplotlib inline

In [0]:
# useful funcs

from keras.layers import Lambda, multiply

def Crop(dim, start, end, **kwargs):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    def func(x):
        dimension = dim
        if dimension == -1:
            dimension = len(x.shape) - 1
        if dimension == 0:
            return x[start:end]
        if dimension == 1:
            return x[:, start:end]
        if dimension == 2:
            return x[:, :, start:end]
        if dimension == 3:
            return x[:, :, :, start:end]
        if dimension == 4:
            return x[:, :, :, :, start:end]

    return Lambda(func, **kwargs)

def cus_acc(y_true, y_pred):
    true = Crop(2,1,-1)(y_true)
    y_hat = Crop(2,1,-1)(y_pred)
    yy = multiply([true, y_hat])

    num_1_yy = K.sum(yy, axis=0)
    # num_2_yy = K.sum(yy, axis=2)
    sum_yy = K.sum(num_1_yy)

    
    num_1_true = K.sum(true, axis=0)
    # num_2_true = K.sum(true, axis=2)
    sum_true = K.sum(num_1_true)
    print(sum_true.shape)

    return sum_yy / sum_true

class WeightedCategoricalCrossEntropy(object):

  def __init__(self, weights):
    nb_cl = len(weights)
    self.weights = np.ones((nb_cl, nb_cl))
    for class_idx, class_weight in weights.items():
      self.weights[0][class_idx] = class_weight
      self.weights[class_idx][0] = class_weight
    self.__name__ = 'w_categorical_crossentropy'

  def __call__(self, y_true, y_pred):
    return self.w_categorical_crossentropy(y_true, y_pred)

  def w_categorical_crossentropy(self, y_true, y_pred):
    nb_cl = len(self.weights)
    final_mask = K.zeros_like(y_pred[..., 0])
    y_pred_max = K.max(y_pred, axis=-1)
    y_pred_max = K.expand_dims(y_pred_max, axis=-1)
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
    for c_p, c_t in product(range(nb_cl), range(nb_cl)):
        w = K.cast(self.weights[c_t, c_p], K.floatx())
        y_p = K.cast(y_pred_max_mat[..., c_p], K.floatx())
        y_t = K.cast(y_pred_max_mat[..., c_t], K.floatx())
        final_mask += w * y_p * y_t
    return K.categorical_crossentropy(y_pred, y_true) * final_mask


In [0]:
# Training 

from seq2seq import AttentionSeq2Seq
import os
import numpy as np
import datetime
import keras.backend as K
from keras.utils.np_utils import to_categorical
from keras.layers import Activation, TimeDistributed, Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
from functools import partial
from itertools import product

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

today = datetime.date.today()
training_set = np.load("/content/drive/My Drive/projects/MWEIs/tmp/train_train_50_2.npy", allow_pickle=True)

input_length = training_set[0][0].shape[0]
input_dim = training_set[0][0].shape[1]
output_length = training_set[0][1].shape[1]
output_dim = 3
hidden_dim = 512

print(input_length)
print(input_dim)
print(output_length)
print(today)

def split(training_set):
    x = list()
    y = list()
    for pair in training_set:
        x.append(pair[0])
        y.append(np.squeeze(to_categorical(pair[1], num_classes=output_dim)))
    x = np.asarray(x)
    y = np.asarray(y)
    return x, y

x, y = split(training_set)
custom_loss = WeightedCategoricalCrossEntropy({0: 1, 1: 25., 2: 50.})

# model configs
mode = "test_mode"
data_type = "test"
epochs = 100
batch_size = 64
depth = 1
figs_path = "/content/drive/My Drive/projects/MWEIs/docs/diagram/model_figs/"
save_to = "/content/drive/My Drive/projects/MWEIs/models/"

model = Sequential()
model.add(
    AttentionSeq2Seq(
        output_dim=output_dim,
        hidden_dim=hidden_dim,
        output_length=output_length,
        input_shape=(input_length, input_dim),
        depth=depth,
        bidirectional=True,
    )
)
model.add(TimeDistributed(Dense(output_dim)))
model.add(Activation("softmax"))


model.compile(
    loss=custom_loss, optimizer="adam", metrics=["accuracy","mae", cus_acc]
)

history = model.fit(x, y, epochs=epochs, batch_size=batch_size)

print(history.history.keys())
plt.plot(history.history["loss"])
# plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.savefig(f"{figs_path}{mode}/{data_type}_{today}_loss.png")

# plt.plot(history.history['mae'])
# plt.title("model mae")
# plt.ylabel("mae")
# plt.xlabel("epoch")
# plt.legend(["train", "test"], loc="upper left")
# plt.savefig('/content/drive/My Drive/projects/MWEIs/docs/diagram/model_figs/tests/test_3_mae.png')

plt.plot(history.history["cus_acc"])
# plt.plot(history.history['val_WinAcc'])
plt.title("model cus_acc")
plt.ylabel("cus_acc")
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(f"{figs_path}{mode}/{data_type}_{today}_cusAcc.png")

plt.show()

model.save_weights(f"{save_to}{mode}/{data_type}_{today}_model_{input_length}.h5")

